In [1]:
import numpy as np
import math

L, d_k, d_v = 4, 8, 8
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L ,d_v)

print("Q :", q)
print(f"K : {k}")
print("V: ", v)

Q : [[-1.39958713 -0.22305316 -0.24340208 -1.08990745  0.53553258 -1.07093933
  -0.41055511  0.33091434]
 [ 0.93078362 -0.52747605  0.24937592 -0.24626193  0.00363866  0.88400743
   0.07809847 -0.95373199]
 [-0.54746032 -0.25787686 -0.41933473  0.19818332 -0.38553762  0.35117474
  -2.03025696 -0.88274714]
 [ 0.75824541 -1.17511898  1.04668314  0.29246406  0.21119868  1.19905575
  -0.90625828  0.75579743]]
K : [[ 2.12356025 -1.47895449  1.65817848 -0.83509451  0.4421548   0.17046466
   1.02055706  1.41988004]
 [ 1.67720882  0.8355391  -0.85573374  0.3876888   0.97649418 -0.12519466
   0.34522402 -1.04967098]
 [ 0.07225795  0.397058    1.22354147  2.46358776  1.43919241 -1.38963153
   0.49021882 -0.08332518]
 [ 2.04920499  0.2789356   0.19126586  1.37597425  0.37469164  0.39046281
  -0.64558209  0.65654841]]
V:  [[ 0.1183661  -1.72408159 -0.13483841 -0.58797534  0.99422244 -0.69578208
   0.10959783  2.20105546]
 [-0.00756153 -0.45394379 -0.61849731 -0.16377034  0.48347382 -0.15590695
  -

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>self attention</mtext>
  <mo>=</mo>
  <mi>s</mi>
  <mi>o</mi>
  <mi>f</mi>
  <mi>t</mi>
  <mi>m</mi>
  <mi>a</mi>
  <mi>x</mi>
  <mrow data-mjx-texclass="ORD">
    <mo minsize="2.047em" maxsize="2.047em">(</mo>
  </mrow>
  <mfrac>
    <mrow>
      <mi>Q</mi>
      <mo>.</mo>
      <msup>
        <mi>K</mi>
        <mi>T</mi>
      </msup>
    </mrow>
    <msqrt>
      <msub>
        <mi>d</mi>
        <mi>k</mi>
      </msub>
    </msqrt>
  </mfrac>
  <mo>+</mo>
  <mi>M</mi>
  <mrow data-mjx-texclass="ORD">
    <mo minsize="2.047em" maxsize="2.047em">)</mo>
  </mrow>
</math>

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>new V</mtext>
  <mo>=</mo>
  <mtext>self attention</mtext>
  <mo>.</mo>
  <mi>V</mi>
</math>

In [3]:
np.matmul(q, k.T)

array([[-2.03055573, -2.5800914 , -1.14248138, -4.21169159],
       [ 2.25366976,  1.73246562, -1.54920081,  1.13902682],
       [-5.07800302, -0.89273901, -2.13136327, -0.27750446],
       [ 5.28551216, -1.54250085, -0.28016198,  3.45723831]])

In [4]:
q.var(), k.var(), np.matmul(q, k.T).var()

(0.5917010369907185, 0.9793615634282073, 6.956340053105443)

In [10]:
scaled = np.matmul(q, k.T) / math.sqrt(d_k)
print(scaled.var()) #the variance for this is significantly smaller
print(scaled)

0.8695425066381802
[[-0.71790986 -0.91220006 -0.40392817 -1.48905784]
 [ 0.79679258  0.61251909 -0.5477252   0.40270679]
 [-1.79534518 -0.31563091 -0.75355071 -0.09811264]
 [ 1.86871075 -0.54535641 -0.09905222  1.22231833]]



Masking

    This is to ensure words don't get context from words generated in the future.
    Not required in the encoders, but required in  the decoders


In [12]:
mask = np.tril(np.ones( (L, L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [13]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [14]:
scaled + mask

array([[-0.71790986,        -inf,        -inf,        -inf],
       [ 0.79679258,  0.61251909,        -inf,        -inf],
       [-1.79534518, -0.31563091, -0.75355071,        -inf],
       [ 1.86871075, -0.54535641, -0.09905222,  1.22231833]])

Softmax application

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>softmax</mtext>
  <mo>=</mo>
  <mfrac>
    <msup>
      <mi>e</mi>
      <mrow data-mjx-texclass="ORD">
        <msub>
          <mi>x</mi>
          <mi>i</mi>
        </msub>
      </mrow>
    </msup>
    <mrow>
      <munder>
        <mo data-mjx-texclass="OP">&#x2211;</mo>
        <mi>j</mi>
      </munder>
      <msubsup>
        <mi>e</mi>
        <mi>j</mi>
        <mi>x</mi>
      </msubsup>
    </mrow>
  </mfrac>
</math>

In [15]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x), axis = -1)).T

In [31]:
attention = softmax(scaled+mask)
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.54593845, 0.45406155, 0.        , 0.        ],
       [0.12156593, 0.53387995, 0.34455413, 0.        ],
       [0.57040106, 0.0510228 , 0.07972449, 0.29885165]])

In [35]:
new_v = np.matmul(attention, v)
new_v

array([[ 0.1183661 , -1.72408159, -0.13483841, -0.58797534,  0.99422244,
        -0.69578208,  0.10959783,  2.20105546],
       [ 0.06118721, -1.14736086, -0.35444932, -0.39536016,  0.76231113,
        -0.45064554, -0.41923784,  1.61325559],
       [ 0.1071705 , -0.54247533, -0.57570776,  0.23574439,  0.10835509,
         0.20141247, -0.64113145,  0.69963236],
       [ 0.21034145, -0.8339612 ,  0.49000993, -0.21182805,  0.42808913,
        -0.37614096, -0.27954744,  0.88531653]])

In [34]:
v

array([[ 0.1183661 , -1.72408159, -0.13483841, -0.58797534,  0.99422244,
        -0.69578208,  0.10959783,  2.20105546],
       [-0.00756153, -0.45394379, -0.61849731, -0.16377034,  0.48347382,
        -0.15590695, -1.05508056,  0.90651759],
       [ 0.28099552, -0.26275775, -0.66495412,  1.14541033, -0.78543672,
         1.07162106, -0.26459835, -0.15066588],
       [ 0.40424365,  0.64770088,  2.17998622,  0.1358282 , -0.33818133,
        -0.18987932, -0.89386833, -1.35321033]])

In [39]:
def sclaed_dot_product_attention(q, k, v , mask = None):
    d_k = q.shape[-1]
    scaled = np.matmul(q, k.T) / math.sqrt(d_k)

    if mask.any():
        scaled += mask
    
    attention = softmax(scaled)
    out = np.matmul(attention, v)
    return out,attention

In [40]:
values, attention = sclaed_dot_product_attention(q, k, v, mask = mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)

Q
 [[-1.39958713 -0.22305316 -0.24340208 -1.08990745  0.53553258 -1.07093933
  -0.41055511  0.33091434]
 [ 0.93078362 -0.52747605  0.24937592 -0.24626193  0.00363866  0.88400743
   0.07809847 -0.95373199]
 [-0.54746032 -0.25787686 -0.41933473  0.19818332 -0.38553762  0.35117474
  -2.03025696 -0.88274714]
 [ 0.75824541 -1.17511898  1.04668314  0.29246406  0.21119868  1.19905575
  -0.90625828  0.75579743]]
K
 [[ 2.12356025 -1.47895449  1.65817848 -0.83509451  0.4421548   0.17046466
   1.02055706  1.41988004]
 [ 1.67720882  0.8355391  -0.85573374  0.3876888   0.97649418 -0.12519466
   0.34522402 -1.04967098]
 [ 0.07225795  0.397058    1.22354147  2.46358776  1.43919241 -1.38963153
   0.49021882 -0.08332518]
 [ 2.04920499  0.2789356   0.19126586  1.37597425  0.37469164  0.39046281
  -0.64558209  0.65654841]]
V
 [[ 0.1183661  -1.72408159 -0.13483841 -0.58797534  0.99422244 -0.69578208
   0.10959783  2.20105546]
 [-0.00756153 -0.45394379 -0.61849731 -0.16377034  0.48347382 -0.15590695
  -1.0